In [5]:
from simhash import Simhash
import jieba
from wordcloud import WordCloud
import os

def load_sougou_content():
    # with open("news_sohusite_content.txt") as F:
    # 测试阶段仅加载前1w条记录
    with open("news_sohusite_xml.dat", encoding="gb18030") as F:
        content = F.readlines()

    return content


def write_content_full():
    content = load_sougou_content()
    with open("passageList.txt", mode="w", encoding="gb18030") as f:
        count = 0
        for i in content:
            if "<content>" in i:
                i = i.replace("</content>", "")
                i = i.replace("<content>", "")
                i = i.replace("\n", "")
                i = i.replace("\u3000", "")
                i = i.replace("\ue40c", "")
                if len(i) <= 100:
                    continue
                f.write(i)
                f.write("\n")
                count += 1
                print(count)


def read_content_full(lines=0):
    with open("passageList.txt", mode="r", encoding="gb18030") as f:
        if lines:
            pl = []
            return [f.readline() for i in range(lines)]
        else:
            lines = f.readlines()
            return lines


def load_stopwords():
    with open("stopwords/cn_stopwords.txt", encoding="utf-8") as F:
        stopwords = F.readlines()
    return [word.strip() for word in stopwords]


def cut_words(passageList):
    if isinstance(passageList, str):
        return [word for word in jieba.cut(passageList) if word not in stopwords]
    else:
        return [
            [word for word in jieba.cut(passgae) if word not in stopwords]
            for passgae in passageList
        ]

def write_cut_list(passageList):
    with open("cutPassageList.txt", mode="r+", encoding="gb18030") as f:
        line = len(f.readlines())
        count = 0
        doTimes = 0
        for news in passageList:
            count+=1
            if count<=line:
                continue
            seq = " ".join(cut_words(news))
            f.write(seq)
            doTimes+=1
        print (doTimes)
            

def read_cut_list(lines=0):
    with open("cutPassageList.txt", mode="r", encoding="gb18030") as f:
        if lines:
            pl = []
            return [f.readline() for i in range(lines)]
        else:
            lines = f.readlines()
            return lines

def generate_wordcloud(indexList, index0, cutPassageList_space):
    os.makedirs(f'results/{index0}')
    indexList1 = []
    indexList1.append(index0)
    indexList1.extend(indexList)
    for index in indexList1:
        news = cutPassageList_space[index]
        wc = WordCloud(
            background_color="white",
            width=1000,
            height=1000,
            font_path="yhw7.ttf",
        ).generate(news)
        wc.to_file(f'results/{index0}/{index}.png')

In [29]:
stopwords = load_stopwords()
passageList = read_content_full()
write_cut_list(passageList)

1093388


In [5]:
stopwords = load_stopwords()
passageList = read_content_full()
write_cut_list(passageList)

In [2]:
cutPassageList_space=[]
cutPassageList = read_cut_list()
cutPassageList_space = [[word for word in line.split()] for line in cutPassageList]
# for i in range(len(cutPassageList)):
#     line = cutPassageList[i]
#     line = line.replace('\n','')
#     seq=[]
#     for word in line.split():
#         seq.append(word)
#     cutPassageList_space.append(seq)

### 进行分词

In [4]:
import pandas
import random
dataFrameList=[]
passageListLen = len(cutPassageList_space)
for i in range(200):
    passageIndex = random.randint(0,passageListLen-1)
    testPassage = cutPassageList_space[passageIndex]
    testNewsHash = Simhash(testPassage)
    simHashList = []
    df = pandas.DataFrame()
    for news in cutPassageList_space:
        hash = Simhash(news)
        score = testNewsHash.distance(hash)
        simHashList.append(score)
    simHashListSorted = sorted(enumerate(simHashList), key=lambda item: item[1])[:20000]


    simList=[]
    indexList=[]
    for news in simHashListSorted:
        index = news[0]
        sim = news[1]
        indexList.append(index)
        simList.append(sim)
    df['index'] = indexList
    df['sim'] = simList
    df.to_csv(f'results/{passageIndex}.csv')


    simList=[]
    indexList=[]
    with open(f'results/{passageIndex}.txt',mode='w+',encoding="gb18030") as f:
        f.write(f'{passageIndex}\n{testPassage}\n\n\n\n')
    
    with open(f'results/{passageIndex}.txt',mode='a+',encoding="gb18030") as f:
        count = 0
        for news in simHashListSorted:
            if count >= 50:
                break
            index = news[0]
            sim = news[1]
            if sim<=3:
                continue
            news = cutPassageList[index]
            indexList.append(index)
            simList.append(sim)
            f.write(f'index:{index} sim:{sim}\n{news}\n\n')
            count+=1
    generate_wordcloud(indexList,passageIndex,cutPassageList)
    

TypeError: 'NoneType' object is not iterable

In [11]:
cutPassageList = read_cut_list(310000)


In [10]:
generate_wordcloud([90792,36576,104922],90792,cutPassageList)

In [12]:
generate_wordcloud([27197,109937,304998],27197,cutPassageList)

In [ ]:
generate_wordcloud([31078,109937,304998],31078,cutPassageList)